In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import random
import os
import glob
from Bio.PDB import *
from Bio.PDB.ResidueDepth import get_surface
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.ResidueDepth import min_dist
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from Bio.PDB.DSSP import DSSP
import shutil
from pyquaternion import Quaternion
import numpy as np
import fnmatch
import copy
import torch
import itertools
from sklearn.neighbors import NearestNeighbors
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from threading import Thread
import statistics
from Bio.PDB.PDBExceptions import PDBConstructionWarning
import warnings
warnings.simplefilter('ignore', PDBConstructionWarning)
p=PDBParser()
amino_acids = {'ALA':'A', 'ARG':'R', 'ASN':'N', 'ASP':'D', 'CYS':'C', 'GLN':'Q', 'GLU':'E', 'GLY':'G', 'HIS':'H' , 'ILE':'I', 'LEU':'L', 'LYS':'K','MET':'M','PHE':'F','PRO':'P','SER':'S','THR':'T','TRP':'W','TYR':'Y','VAL':'V'}

polarity = {'ALA':  8.100,  'ARG': 10.500,  'ASN': 11.600,  'ASP': 13.000,  'CYS':  5.500 , 'GLN': 10.500,  
'GLU': 12.300 , 'GLY':  9.000 , 'HIS': 10.400 , 'ILE':  5.200 , 'LEU':  4.900,  'LYS': 11.300,  
'MET':  5.700,  'PHE':  5.200,  'PRO':  8.000,  'SER':  9.200 , 'THR':  8.600,  'TRP':  5.400,  
'TYR':  6.200,  'VAL':  5.900 }

C:\Users\Squis\.conda\envs\BioPy\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!msms

MSMS 2.6.1 started on Local PC
Copyright M.F. Sanner (1994)
Compilation flags 
MSMS: No input stream specified


In [3]:
p=PDBParser()
amino_acid_1 = (['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS' , 'ILE', 'LEU', 'LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL'])

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(amino_acid_1)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [4]:
def one_hot(res):
    if res in amino_acid_1:
        return onehot_encoded[amino_acid_1.index(res)]    

In [5]:
def generate_edges(len_whole_rec_feat, len_whole_lig_feat):
    whole_edges_out_full =[]
    a=list(range(len_whole_rec_feat))
    b=list(range(len_whole_rec_feat, len_whole_rec_feat+len_whole_lig_feat))
    tol=list(itertools.product(a, b))
    tot=[list(x) for x in tol]
    whole_edges_out_full.append(tot)
    return whole_edges_out_full

In [6]:
def input_processing(filen,ind):
    structure=p.get_structure('2bth',filen)
    whole_feat=[]
    amino_acid=[]
    coordinate = []
    depth = []
    basis_vectors= []
    gravy = []
    polaritya = []
    knearest_percentage = []
    OneHot=[]
    ADJ=[]
    drop=[]
    index=ind
    drop_ind=[]
    for model in structure:
        surface = get_surface(model)
        for chain in model:
            for residue in chain:
                amino_acid.append(amino_acids[residue.get_resname()])
                for atom in residue:
                    if atom.get_name() in ['CA']:
                        OneHot.append(one_hot(residue.get_resname()))
                        ca_coord= atom.get_coord()
                coordinate.append(ca_coord.tolist())
                depth.append(min_dist(ca_coord, surface))
                gravy.append(ProteinAnalysis(amino_acids[residue.get_resname()]).gravy())
                polaritya.append(polarity[residue.get_resname()])
    X = np.array(coordinate)
    distance=torch.cdist(torch.tensor(coordinate),torch.tensor(coordinate))
    dist=torch.where(distance<=10,1,0)
    ADJ.append(dist)
    knn = NearestNeighbors(n_neighbors=4)
    knn.fit(X)
    avg_dp=statistics.median(depth)
    def neighborhood_percentage(x):
        neig = knn.kneighbors(x, return_distance=False)
        se =[amino_acid[i] for i in neig[0]]
        seq=''
        seq=seq.join(se)
        percentage = [ProteinAnalysis(seq).get_amino_acids_percent()[i] for i in list(amino_acids.values())]
        return percentage
    for i in range(len(X)):
        knearest_percentage.append(neighborhood_percentage([X[i]]))
    assert len(coordinate) == len(depth)==len(gravy)==len(polaritya)==len(knearest_percentage)
    for i in range(len(coordinate)):
    
        if depth[i]<avg_dp:
            drop_ind.append(ind+len(drop))
            drop.append(coordinate[i])
            continue
        ind=ind+1
        whole_feat.append(coordinate[i]+[depth[i]]+[gravy[i]]+ [polaritya[i]]+ knearest_percentage[i]+OneHot[i].tolist())
    return whole_feat,drop,drop_ind

In [7]:
def pdbgen(recc,ligc,filename):
    rec = open('receptor1.pdb','w')
    lig = open('ligand1.pdb','w')
    inp = open(filename, 'r')
    x1,x2,x3 = torch.from_numpy(np.random.uniform(3.14/12, 3.14, 3))
    cos1=torch.cos(x1)
    cos2=torch.cos(x2)
    cos3=torch.cos(x3)
    sin1=torch.sin(x1)
    sin2=torch.sin(x2)
    sin3=torch.sin(x3)
    RMatrix=torch.tensor([[cos1*cos2*cos3-sin1*sin3,cos3*cos2*sin1+sin3+cos1,-cos3*sin2 ],\
                            [-sin3*cos2*cos1-cos3*sin1, -sin3*cos2*sin1+cos3*cos1, sin3*sin2],\
                            [sin2*cos1,sin2*sin1,cos2]])
    #t = torch.from_numpy(np.random.uniform(8.00, 30.0, 3))
    rotmat = torch.tensor([x1,x2,x3]) # RMatrix.flatten()
    outfeat = rotmat # torch.cat((t, rotmat),0)
    l=['']*11
    #print(outfeat)
    count=0
    for line in inp:
        if line[:4]=='ATOM' and line.split()[2]=='CA':
                li=line.split()
                if line[21] in recc:
                    rec.write(line)
                    count+=1
                    #count+=1
                elif line[21] in ligc:#if feat_in[count][23]==1 and feat_in[count][24]==0:
                    l[0]=li[0].ljust(4)
                    l[1]=str(count).rjust(6)
                    l[2]=li[2].ljust(3)
                    if len(li[3])>3 and li[3][1]!='A' :
                        continue
                    elif len(li[3])>3 and li[3][1]=='A':
                        li[3]=li[3][1:]
                    l[3]=li[3].ljust(3)
                    l[4]=line[21]
                    l[5]=('%4d' % (int(count))).rjust(4)
                    coordinates = [float(line[29:38]), float(line[38:46]), float(line[46:54])]
                   # cood = ((RMatrix @ torch.tensor(coordinates).double().T)).T
                    l[6]=('%8.3f' % (coordinates[0])).rjust(8)
                    l[7]=('%8.3f' % (coordinates[1])).rjust(8)
                    l[8]=('%8.3f' % (coordinates[2])).rjust(8)
                    l[9]=('%6.2f' % (float(line[54:60]))).rjust(6)
                    l[10]=('%6.2f' % (float(line[60:66]))).ljust(6)
                    lig.write('{0} {1}  {2} {3} {4}{5}    {6}{7}{8}{9}{10}' .format(l[0],l[1],l[2],l[3],l[4],l[5],l[6],l[7],l[8],l[9],l[10]))
                    lig.write('\n')
                    count+=1
    rec.close()
    lig.close()
    inp.close()
    return outfeat

In [8]:
dataset=[]
sumin=0
sumout=0
datafol='data'
samplelist=[]#zero_pad = [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0., 0.,0.,0.,0,0]
def prepfull(filename):
    full = open('fullcomplex.pdb','w')
    inp = open(filename, 'r')
    count=1
    l=['']*11
    for line in inp:
        if line[:4]=='ATOM' and line.split()[2]=='CA':
                    li=line.split()
                    l[0]=li[0].ljust(4)
                    l[1]=str(count).rjust(6)
                    l[2]=li[2].ljust(3)
                    if len(li[3])>3 and li[3][1]!='A' :
                        continue
                    elif len(li[3])>3 and li[3][1]=='A':
                        li[3]=li[3][1:]
                    l[3]=li[3].ljust(3)
                    l[4]=line[21]
                    l[5]=('%4d' % (int(count))).rjust(4)
                    cood = [float(line[29:38]), float(line[38:46]), float(line[46:54])]
                    l[6]=('%8.3f' % (cood[0])).rjust(8)
                    l[7]=('%8.3f' % (cood[1])).rjust(8)
                    l[8]=('%8.3f' % (cood[2])).rjust(8)
                    l[9]=('%6.2f' % (float(line[54:60]))).rjust(6)
                    l[10]=('%6.2f' % (float(line[60:66]))).ljust(6)
                    full.write('{0} {1}  {2} {3} {4}{5}    {6}{7}{8}{9}{10}' .format(l[0],l[1],l[2],l[3],l[4],l[5],l[6],l[7],l[8],l[9],l[10]))
                    full.write('\n')
                    count+=1
    full.close()
    inp.close()

In [27]:
def outedgegen(chain0,chain1,filename,drop):
    output_edge =[]
    structure=p.get_structure('2bth',filename)
    rec_coordinate=[]
    lig_coordinate=[]
    edgesout = []
    ind=0
    index=[]
    for model in structure:
        for chain in model:
            if chain.id in chain0:
                for residue in chain:
                    for atom in residue:
                        if atom.get_name() in ['CA']:
                            ind+=1
                            if atom.get_coord().tolist() in drop:
                                index=index+[ind]
                                continue
                            rec_coordinate.append(atom.get_coord().tolist())
            if chain.id in chain1:
                for residue in chain:
                    for atom in residue:
                        if atom.get_name() in ['CA']:
                            ind+=1
                            if atom.get_coord().tolist() in drop:
                                index=index+[ind]
                                continue
                            lig_coordinate.append(atom.get_coord().tolist())
    #'''
    dis=torch.cdist(torch.tensor(rec_coordinate), torch.tensor(lig_coordinate))
    #print(len(depth), len(rec_coordinate), len(lig_coordinate))
    rec_c = len(rec_coordinate)
    tot = rec_c+ len(lig_coordinate)
    b = dis <=10
    indices = b.nonzero()
    readyToPair = torch.zeros(tot)
    readyToPair[indices[:,0]] = 1
    readyToPair[indices[:,1]+rec_c] = 1
    return readyToPair


In [30]:

files =glob.glob('data/data/prism/3AAE_D_H.pdb')
files.sort()
index=0
c=1

samples=[]
for filename in files:
    flag=0
    print(filename)
    chain=filename.split('.')[0].split('_') 
    try:
        torch.load(('feat/All-down/All_down/pssm',chain[0][-4:]+'.'+chain[1]+'.'+chain[2]+'.'+'_pssm.pt'))
        flag=1
    except:
        pass
    if flag==1:
        continue


#     chain=filename.split('.')[1].split('_')[1:]
    try:
        ps=torch.load('pssm/'+chain[0][16:]+'_'+chain[1]+'_'+chain[2]+'_pssm.pt')
    except:
        continue
    prepfull(filename)
    output_feat = pdbgen(list(chain[1]),list(chain[2]),  filename)
   # try:
    receptor_feat, drop_r ,drop_ind_r= input_processing('receptor1.pdb',index)
    rec_flag = [0]*len(receptor_feat)
    for i in range(len(receptor_feat)):
        receptor_feat[i] = receptor_feat[i]+[rec_flag[0]]
    ligand_feat, drop_l,drop_ind_l = input_processing('ligand1.pdb',index)
    for i in range(len(ligand_feat)):
        ligand_feat[i] = ligand_feat[i]+[1]
    receptor_feat.extend(ligand_feat)
    drop=drop_l+drop_r

    readyToPair = outedgegen(list(chain[1]),list(chain[2]),  'fullcomplex.pdb',drop)
    drop_ind=drop_ind_r
    last=drop_ind_r[len(drop_ind_r)-1]
    for i in drop_ind_l:
        drop_ind=drop_ind+[i+last]
        

    j=0
    i=0
    try:
        for i in drop_ind:
            ps=np.delete(ps,(i-j),axis=0)
            j+=1
    except:
        continue
    if ps.shape[0]!=len(receptor_feat):
        continue
    print(readyToPair)
#     torch.save(readyToPair, os.path.join('feat/All-down/All_down/feat',chain[0][-4:]+'.'+chain[1]+'.'+chain[2]+'.'+'_interface.pt'))
#     torch.save(torch.tensor(receptor_feat),os.path.join('feat/All-down/All_down/feat',chain[0][-4:]+'.'+chain[1]+'.'+chain[2]+'.'+'_input_feat.pt'))
#     torch.save((ps),os.path.join('feat/All-down/All_down/pssm',chain[0][-4:]+'.'+chain[1]+'.'+chain[2]+'.'+'_pssm.pt'))
    print(c)
    c+=1
#    # except:
    

data/data/prism/3AAE_D_H.pdb
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.,

In [77]:
pssm=glob.glob('pssm/3AEU_C_B.pt')
pot=torch.load('pssm/3AEU_C_B_pssm.pt')
pot=pot.numpy()
print(pot.shape)



(831, 20)


In [2]:
files =glob.glob('data/data/prism/*.pdb')
files.sort()
ccc=0
samples=[]
for filename in files[4:5]:
    chain=filename.split('.')[0].split('_') 
    print(chain)

['data/data/prism\\2ZWI', 'A', 'B']


In [ ]:
chain

In [ ]:
filename.split('_')[0][-4:]